In [1]:
import sys
import datetime
import tempfile
import re
import math
import json
from easyocr import Reader
import cv2
from PIL import Image
import imutils
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
from pytesseract import Output

In [2]:
cv2.__version__

'4.5.4'

In [3]:
pytesseract.get_tesseract_version() 
# pytesseract.__version__

<Version('4.1.1')>

In [8]:
image_path = '../output/수입면장-PDF-img9.jpg'

x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin

cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),15)

In [19]:
image = cv2.imread(image_path)
draw_image = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, imthres = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

In [22]:
# custom_config = r'-l kor+eng --psm 6'
custom_config = r'-l kor+eng --psm 4'
text = pytesseract.image_to_string(gray, config=custom_config)

In [23]:
text

" \n        \n\n“~  m=\n수입신고필증\n\n[2 세관.과             (9 입항일\n\n※ 처리기\n\n                                                                                                                         \n\n   \n  \n\n   \n\n \n\n2 전자인보이스 제출번호\n\n \n\n간 : 3잎\n\n  \n  \n\n \n\n     \n                                                                                                                   \n\n      \n\n  \n\n     \n      \n \n\n \n\n  \n\n \n\n     \n  \n\n \n\n    \n \n\n \n\n                                                                                                                                                                                                                                                                     \n\n \n\n                                                                                                                                                                                                                                                          

In [17]:
tel = re.findall(r'(?:Tel )([\+\(]?[0-9][0-9 .\-\(\)]{8,}[0-9])', text)[0]
mobile = re.findall(r'(?:Mobile )([\+\(]?[0-9][0-9 .\-\(\)]{8,}[0-9])', text)[0]
emails = re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", text)[0]
addr = re.findall(r"[0-9\.\-+_]+\,.*", text)[0]
print("유선전화 : {}".format(tel))
print("휴대전화 : {}".format(mobile))
print("이메일 : {}".format(emails))
print("주소 : {}".format(addr))

IndexError: list index out of range

In [24]:
from easyocr import Reader

langs = ['ko', 'en']
 
print("[INFO] OCR'ing input image...")
reader = Reader(lang_list=langs, gpu=False)
results = reader.readtext(image)

Using CPU. Note: This module is much faster with a GPU.


[INFO] OCR'ing input image...


/home/codev/Dev/ocr-service/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  warnings.warn(
/home/codev/Dev/ocr-service/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/codev/Dev/ocr-service/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [27]:
results

[([[195, 101], [305, 101], [305, 145], [195, 145]],
  '점환 인',
  0.20221127569675446),
 ([[300, 144], [330, 144], [330, 174], [300, 174]], '떼', 0.04825252492537402),
 ([[194, 172], [300, 172], [300, 220], [194, 220]], '미컴증', 0.2933442004388921),
 ([[289, 229], [431, 229], [431, 269], [289, 269]],
  '(yifi-P =',
  0.07947292749917945),
 ([[656, 223], [1048, 223], [1048, 285], [656, 285]],
  '수 입 신 고 필 증',
  0.6978443539982948),
 ([[1252, 266], [1362, 266], [1362, 292], [1252, 292]],
  'X 처리기간',
  0.9655893874660583),
 ([[1372, 266], [1408, 266], [1408, 292], [1372, 292]],
  '3일',
  0.9973745033290212),
 ([[284, 292], [368, 292], [368, 320], [284, 320]],
  '신고번호',
  0.9998964071273804),
 ([[534, 294], [600, 294], [600, 324], [534, 324]], '신고입', 0.7856468072001938),
 ([[752, 298], [820, 298], [820, 324], [752, 324]],
  '세관 과',
  0.5537016451458531),
 ([[982, 298], [1046, 298], [1046, 328], [982, 328]],
  '입합일',
  0.9988258602715305),
 ([[1168, 300], [1376, 300], [1376, 330], [1168, 330]],


In [26]:
simple_results = reader.readtext(image, detail = 0, paragraph=True)
simple_results

["점환 인 떼 미컴증 (yifi-P = 수 입 신 고 필 증 0G시집인 X 처리기간 3일 신고번호 신고입 세관 과 입합일 전자인보이스 제출번호 44025-22-030378M 2022/03/28 030 C3 2022/03/28 B/L(AWB)번호 화물관리번호 반입일 징수형태 YHBUSo1oz2 2ZCKCO02631-3024-00D1 11 신 고 인 이정과세법인 나병인 물관계회 @)원산지중명서 l 총중량 수 입 자 한국센트럴글래스(주)한국센트-1-12-1-01-6 A) 입히전신고 유무 N A,000 KG 납세의무 사 (인국신트-1-12-1-01-6 220-88-30353) 신고구분 0 가려신고서 콩노상셋수 (주소) 서율 강남구 역심동 태히란로116 동경빌딩요층 (06120) 일반서류신고 유무 Y 5 GT (상호) 한국센트럭글래스(주) @@ 거래구분 11 7) 국내도착항 KRPUS 운송형태 (성명) KAWAKAMI MAS 일반형태수입 부산함 10-FC 요송 주선 인 국보해운 주식회사 (KBEC) @종류 21 적손국 JP JAPAN 무역거래처 CENTRAL GLASs CO LTD (JP) JPCENTRAOUISN 일반수입내수용) 선기명 CONMAR MOON AG MASTER B/L버호 PCLUONY2D8OII 운수기관부호 @검사(반입)장소 03012240- (한국히지스터미널(주)부산컨테) 품명 . 규려 (관번회종관수 001/002 $품 명 CHEMICAL PREPARATIONS 상표 NO @거래압명 CHEMICAL PREPARATIONS '@모댁 . 규격 (@) 성분 수량 단가(USD) 금액(USD) (NO 01) PPROPYLENE GLYCOL PPK-HUZ 14DRUM(S) CAS NO.108-65-6 IMETHYLETHER ACETATE 2,880 KG 40,8 117,504 199,48%",
 '@@세번부호',
 '3824.29-3090| 순중량 117,655K)수 량 144,409,289 환급문량',
 '2,880 KG',
 "'디s검사 5   생락 검사변